# PT-Flash Calculator
## ChE 3210: Phase Equilibria
### Martin Colahan

Please run the following cells by entering the cell and hitting shift + enter.

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from scipy.optimize import brentq
from enum import IntEnum

spec_names = ['CO2','H2S','N2','Methane','Ethane','Propane','i-Butane', 'n-Butane','i-Pentane','n-Pentane', 'n-Hexane', 'n-Heptane','n-Octane','n-Nonane','n-Decane']

Tc = [304.2, 373.2, 126.2, 190.56, 305.3, 369.8, 408.1, 425.1, 460.4, 469.6, 504.7, 540.2, 568.8, 594.6, 617.6]
Pc = [73.76, 89.42, 34.0, 45.99, 48.72, 42.48, 36.48, 37.96, 33.84, 33.74, 29.69, 27.36, 24.82, 22.9, 21.08]
omega = [0.225, 0.1, 0.038, 0.012, 0.1, 0.152, 0.181, 0.2, 0.227, 0.251, 0.296, 0.351, 0.394, 0.444, 0.49]

kij = np.array([
    [0.   , 0.134, 0.017, 0.952, 0.169, 0.121, 0.115, 0.115, 0.117, 0.119, 0.125, 0.133, 0.14 , 0.148, 0.155],
    [0.134, 0.   , 0.   , 0.101, 0.088, 0.08 , 0.   , 0.073, 0.   , 0.065, 0.059, 0.05 , 0.044, 0.036, 0.029],
    [0.017, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.952, 0.101, 0.   , 0.   , 0.003, 0.011, 0.   , 0.019, 0.   , 0.026, 0.032, 0.036, 0.039, 0.041, 0.043],
    [0.169, 0.088, 0.   , 0.003, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.121, 0.08 , 0.   , 0.011, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.115, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.115, 0.073, 0.   , 0.019, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.117, 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.119, 0.065, 0.   , 0.026, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.125, 0.059, 0.   , 0.032, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.133, 0.05 , 0.   , 0.036, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.14 , 0.044, 0.   , 0.039, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.148, 0.036, 0.   , 0.041, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
    [0.155, 0.029, 0.   , 0.043, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]
  ])

R = 0.08314  # bar*L/mol*K

class PhaseTypes(IntEnum):
    Vapor = 0
    Liquid = 1


def PR_Z(T, P, species, x, identify_phase=False):
    assert len(species) == len(x)
    n_spec = len(species)
    
    spec_indices = [spec_names.index(spec) for spec in species]
    
    a = []
    b = []
    for i in spec_indices:
        kappa = 0.37464 + 1.54226 * omega[i] - 0.26992 * omega[i] ** 2
        alpha = (1 + kappa * (1 - np.sqrt(T / Tc[i]))) ** 2
        b.append(0.07780 * R * Tc[i] / Pc[i])
        a.append(0.45724 * R ** 2 * Tc[i] ** 2 / Pc[i] * alpha)

    a_mix = 0
    b_mix = 0
    for i in range(n_spec):
        for j in range(n_spec):
            aij = np.sqrt(a[i] * a[j]) * (1 - kij[spec_indices[i], spec_indices[j]])
            a_mix += x[i] * x[j] * aij
        b_mix += x[i] * b[i]

    A = a_mix * P / R ** 2 / T ** 2
    B = b_mix * P / (R * T)
    coeffs = [
        1,
        -(1 - B),
        A - 3 * B ** 2 - 2 * B,
        -A * B + B ** 2 + B ** 3
    ]
    roots = np.roots(coeffs)
    if identify_phase:
        Z_list = sorted([np.real(roots[i]) for i in range(len(roots)) if roots[i] > B and np.isreal(roots[i])])
        V = np.array(Z_list) * R * T / P
        ptypes = [V[i]/b_mix < 1.75 for i in range(len(Z_list))]
        return Z_list, ptypes

    else:
        return sorted([np.real(roots[i]) for i in range(len(roots)) if roots[i] > B and np.isreal(roots[i])])
    
def calc_ln_fug_coeffs(T, P, x, species, Z=np.nan, ptype=PhaseTypes.Vapor):
    
    if pd.isna(Z):
        Zlist = PR_Z(T, P, species, x)
        if ptype == PhaseTypes.Vapor:
            Z = max(Zlist)
        elif ptype == PhaseTypes.Liquid:
            Z = min(Zlist)
    
    spec_indices = [spec_names.index(spec) for spec in species]
    n_spec = len(species)
    
    a = []
    b = []
    for i in spec_indices:
        kappa = 0.37464 + 1.54226 * omega[i] - 0.26992 * omega[i] ** 2
        alpha = (1 + kappa * (1 - np.sqrt(T / Tc[i]))) ** 2
        b.append(0.07780 * R * Tc[i] / Pc[i])
        a.append(0.45724 * R ** 2 * Tc[i] ** 2 / Pc[i] * alpha)

    a = np.array(a)
    b = np.array(b)

    A = a * P / R ** 2 / T ** 2
    B = b * P / (R * T)

    a_mix = 0
    b_mix = 0
    aij = np.zeros([n_spec, n_spec])
    for i in range(n_spec):
        for j in range(n_spec):
            aij[i, j] = np.sqrt(a[i] * a[j]) * (1 - kij[spec_indices[i], spec_indices[j]])
            a_mix += x[i] * x[j] * aij[i,j]
        b_mix += x[i] * b[i]

    A_ij = aij * P / R ** 2 / T ** 2
    A_mix = a_mix * P / R ** 2 / T ** 2
    B_mix = b_mix * P / (R * T)

    lnfug_coeff = np.zeros(n_spec)
    for i in range(n_spec):
        aij_sum = 0
        for j in range(n_spec):
            aij_sum += x[j] * A_ij[i,j]

        lnfug_coeff[i] = (B[i] / B_mix * (Z - 1) - np.log(Z - B_mix) - 
                            A_mix / (2 * np.sqrt(2) * B_mix) * (2 * aij_sum / A_mix - B[i]/B_mix) * 
                            np.log((Z + (1 + np.sqrt(2))*B_mix)/(Z + (1 - np.sqrt(2))*B_mix)))

    return lnfug_coeff

def wilson_K(T, P, spec):
    i = spec_names.index(spec)
    return Pc[i] / P * np.exp(5.37 * (1 + omega[i]) * (1 - Tc[i] / T))

def stability_analysis(T, P, species, z):
    #PR_Z(T, P, species, x)
    #calc_ln_fug_coeffs(T, P, x, species, Z=np.nan, ptype=PhaseTypes.Vapor)

    n_spec = len(species)
    # get z props:
    Z_z = PR_Z(T, P, species, z)
    if len(Z_z) > 1:

        # find the Z that produces the lowest gibbs energy
        ln_fug_coeffs = []
        for Z_val in Z_z:
            ln_fug_coeffs.append(calc_ln_fug_coeffs(T, P, z, species, Z=Z_val))

        # G[0] - G[1] = RT*sum(z(ln(phi2) - ln(phi1)))
        gibbs_diff = R * T * (z * (ln_fug_coeffs[1] - ln_fug_coeffs[0])).sum()
        if gibbs_diff < 0:
            # G[0] < G[1]
            Z_z = Z_z[0]
        else:
            Z_z = Z_z[1]
    else:
        Z_z = Z_z[0]

    ln_fug_coeffs_z = calc_ln_fug_coeffs(T, P, z, species, Z=Z_z)

    # d = np.log(z) + ln_fug_coeffs_z

    # Determination of Trial Phases:
    K = np.array([wilson_K(T, P, spec) for spec in species])
    # PR = np.array([P / comp.Pc for comp in components])
    # TR = np.array([T / comp.Tc for comp in components])
    # omega = np.array([comp.omega for comp in components])

    #           # Wilson Equation
    # K[:] = 1 / (PR * np.exp(5.373 * (1 + omega) * (1 - TR)))

    # initial values for stationary points hunt
    X0 = []
    X0.append(K * z)
    X0.append(z / K)

    max_val = 0.999
    for i in range(n_spec):
        molefracs = np.ones(n_spec) * ((1 - max_val) / (n_spec - 1))
        molefracs[i] = max_val
        X0.append(molefracs)

    index = 0
    stability_list = []
    stab_sums = []
    for X in X0:
        rerun = True
        iter_count = 0
        break_criteria = 1e-14

        while True:
            if iter_count > 200:
                raise ValueError("Too many iterations")

            iter_count += 1

            x = X / X.sum()
            X_old = X

            Z_x = PR_Z(T, P, species, x)
            if len(Z_x) > 1:
                # find the Z that produces the lowest Gibbs energy
                ln_fug_coeffs = []
                for Z_val in Z_x:
                    ln_fug_coeffs.append(calc_ln_fug_coeffs(T, P, x, species, Z=Z_val))
                    
                # G[0] - G[1] = RT*sum(z(ln(phi2) - ln(phi1)))
                gibbs_diff = (
                    R * T * (x * (ln_fug_coeffs[1] - ln_fug_coeffs[0])).sum()
                )
                if gibbs_diff < 0:
                    # G[0] < G[1]
                    Z_x = Z_x[0]
                else:
                    Z_x = Z_x[1]
            else: 
                Z_x = Z_x[0]

            ln_fug_coeffs_x = calc_ln_fug_coeffs(T, P, x, species, Z=Z_x)

            X = z * np.exp(ln_fug_coeffs_z) / np.exp(ln_fug_coeffs_x)
            rel_error = abs(X - X_old) / X

            if np.all(rel_error < break_criteria):
                break

        # Post-convergence processing:
        # check if x == z (trivial solution)
        # x = X / X.sum()
        error_to_z = abs(X - z) / z

        if (error_to_z < 1e-9).all():
            # converged to trivial solution
            stability_list.append("Trivial")
            continue

        # stable?
        fudge_factor = 1e-5
        if X.sum() > (1 + fudge_factor):
            # phase is unstable
            stability_list.append(False)
            stab_sums.append(X.sum())

        else:
            stab_sums.append(X.sum())
            stability_list.append(True)

        index += 1

    stability = False not in stability_list 
    return stability

def PT_flash(T, P, species, z):

    assert len(species) == len(z)
    n_spec = len(species)
    
    z = np.array(z)

    single_phase_stable = stability_analysis(T, P, species, z)
    if single_phase_stable:
        Zlist, ptypes = PR_Z(T, P, species, z, True)
        assert len(Zlist) < 2
        if ptypes[0] == PhaseTypes.Vapor:
            V = 1
            y = z.copy()
            x = None
            return V, x, y


    spec_indices = [spec_names.index(spec) for spec in species]

    K = np.array([wilson_K(T, P, spec) for spec in species])
    y = np.ones(n_spec)
    x = np.ones(n_spec)
    V = 0.5

    rashford_rice = lambda V: np.sum(z * (K - 1) / (1 + V * (K - 1)))

    iter_count = 0
    while True:
        iter_count += 1
        assert iter_count < 200

        V = brentq(rashford_rice, 0, 1)
        x = z / (1 + V * (K - 1))
        y = x * K

        liq_fug_coeffs = np.exp(calc_ln_fug_coeffs(T, P, x, species, ptype=PhaseTypes.Liquid))
        vap_fug_coeffs = np.exp(calc_ln_fug_coeffs(T, P, y, species, ptype=PhaseTypes.Vapor))

        liq_fug = liq_fug_coeffs * x * P
        vap_fug = vap_fug_coeffs * y * P

        errors = abs(liq_fug / vap_fug - 1)
        if np.all(errors < 1E-8):
            break
        else:
            K *= liq_fug / vap_fug 

    return V, x, y

# PT_flash(298.15, 10, ["CO2", "Methane", "Propane", "n-Octane"], [0.1, 0.3, 0.3, 0.3])

In [ ]:
T_widg = widgets.Text(
    value=str(298.15),
    placeholder='Enter Temperature:',
    description='Temp. (K): ',
    disabled=False
)

P_widg = widgets.Text(
    value=str(10),
    placeholder='Enter Pressure',
    description='Pres. (bar): ',
    disabled=False
)

mol_fracs_label = widgets.Label(value="z_i Composition (mole frac.)")

mol_frac_sum_label = widgets.Label()
mole_frac_widgets = []
def on_mole_fracs_update(change):
    z_sum = 0
    for widg in mole_frac_widgets:
        val = widg.value
        try:
            val = float(val)
        except:
            continue
        z_sum += val
    mol_frac_sum_label.value = f'Mole Frac Sum: {round(z_sum, 3)}'

for spec in spec_names:
    z_widg = widgets.Text(
        value='0',
        placeholder='0',
        description=f'{spec}: ',
        disabled=False
    )
    z_widg.observe(on_mole_fracs_update, names='value')
    mole_frac_widgets.append(z_widg)

on_mole_fracs_update(None)


output_widg = widgets.Output()

def on_calculate(b):
    z = []
    species = []
    i = 0
    for i in range(len(spec_names)):
        widg = mole_frac_widgets[i]
        val = widg.value
        try:
            val = float(val)
            if val > 0:
                z.append(val)
                species.append(spec_names[i])
                
        except:
            continue
    
    z = np.array(z)
    z_sum = np.sum(z)
    if z_sum < 0.95 or z_sum > 1.05:
        raise ValueError("Mole frac sum out of bounds.")
    z = z / z_sum

    T = float(T_widg.value)
    P = float(P_widg.value)
    V, x, y = PT_flash(T, P, species, z)
    L = 1 - V
    df = pd.DataFrame()
    df.loc['Quality', 'Vapor'] = V
    df.loc['Quality','Liquid'] = L
    for i in range(len(species)):
        df.loc[species[i], 'Vapor'] = x[i]
        df.loc[species[i], 'Liquid'] = y[i]

    output_widg.clear_output()
    with output_widg:
        display(df)
            

calc_btn = widgets.Button(description='Calculate')
calc_btn.on_click(on_calculate)
    

misc_vbox = widgets.VBox([T_widg, P_widg, mol_fracs_label])
mole_frac_vbox = widgets.VBox(mole_frac_widgets)
inputs_vbox = widgets.VBox([misc_vbox, mole_frac_vbox, mol_frac_sum_label, calc_btn])
main_widg = widgets.HBox([inputs_vbox, output_widg])

display(main_widg)

In [ ]:
T = 450  # K
P = 10   # bar

composition = {
    'CO2':       0.6,
    'H2S':       0,
    'N2':        0,
    'Methane':   0,
    'Ethane':    0,
    'Propane':   0,
    'i-Butane':  0, 
    'n-Butane':  0,
    'i-Pentane': 0,
    'n-Pentane': 0, 
    'n-Hexane':  0, 
    'n-Heptane': 0,
    'n-Octane':  0.4,
    'n-Nonane':  0,
    'n-Decane':  0 
}

assert sum(composition.values()) == 1, "The sum of the overall mole fractions (z_i) must equal 1."

species = []
z = []
for key, val in composition.items():
    if val > 0:
        species.append(key)
        z.append(val)

PT_flash(T, P, species, z)